In [1]:
from Bio.Blast import NCBIXML 
from Bio.Blast import NCBIWWW
import requests
import re
from Bio import SeqIO
from Bio import Entrez
from Bio import Medline
from tqdm import tqdm
from Bio import SearchIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from bs4 import BeautifulSoup
from Bio.SeqFeature import CompoundLocation
import mysql.connector as SQLC
import mysql.connector

In [27]:
import mysql.connector as SQLC
class DB:
#database connection and extraction of sequences and id_genebanks
    def __init__(self, password):        
        if password != "20221207":
            raise Exception("The password is wrong!")

        else:
            self.cnx = SQLC.connect(
                    host ="geo.di.uminho.pt",
                    user ="bioinformatica",
                    password = password, #"20221207"
                    database ="AP_db_KRG")
            self.sequences = []
            self.ids = []
    def extract_sequences(self):
        cursor = self.cnx.cursor()
        cursor.execute("SELECT sequence FROM Gene")
        self.sequences = cursor.fetchall()
        cursor.close()
        return self.sequences
        
    def extract_ids(self):
        cursor = self.cnx.cursor()
        cursor.execute("SELECT ID_genebank FROM Gene")
        self.ids = cursor.fetchall()
        cursor.close()
        return self.ids

In [180]:
DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True 
Cursor = DataBase.cursor()

num= input("Escolha o tamanho máximo das seq: ")
Seq_and_length=[]
just_seq=[]
try:
    sql_Seq_length= f"select Gene.ID_genebank, Gene.length from Gene where Gene.length <{num}"
    Cursor.execute(sql_Seq_length)
    for row in Cursor:
        Seq_and_length.append(str(row)) 
    
    sql_Seq= f"select Gene.ID_genebank from Gene where Gene.length <{num}"
    Cursor.execute(sql_Seq)
    for row in Cursor:
        just_seq.append(str(row))
        
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
    
print (Seq_and_length)

if just_seq == []:
        print()
        print('nenhuma seq inferior a esse tamanho, pesquise de novo.')

Escolha o tamanho máximo das seq: 50
["('AAADK0043071.1', 21)", "('AAADS0003348.1', 20)", "('AAADW0009410.1', 21)", "('AAADX0043451.1', 21)", "('ABAAM0346030.1', 20)"]


In [133]:
list_seq=[]
choose= input('escolha a primeira seq com que quer trabalhar: ')
inter= int(choose)
extr= ', '.join(just_seq)
h= extr.replace("(",'')
hh= h.replace(",)",'')
ID_al= hh.split(', ')
select= ID_al[inter]
list_seq.append(select)

# choose2= input('escolha a segunda seq com que quer trabalhar: ')
# inter2= int(choose2)
# extr2= ', '.join(just_seq)
# h2= extr2.replace("(",'')
# hh2= h2.replace(",)",'')
# ID_al2= hh2.split(', ')
# select2= ID_al2[inter2]
# list_seq.append(select2)

limpar_aspas_all= ', '.join(ID_al)
retirar_all= limpar_aspas_all.replace("'","")
otput_final_all= retirar_all.split(', ')

limpar_aspas= ', '.join(list_seq)
retirar= limpar_aspas.replace("'","")
otput_final= retirar.split(', ')


#cria lista com todos as seq com o tamanho máximo definido:
todas_as_seq=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= otput_final_all
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    seqs= (info.seq)
    todas_as_seq.append(seqs)
    
choose2= input('tipo de molécula com que está a trabalhar: ')    
#cria lista só com as duas seq escolhidas para o alinhamento 
seq_for_al=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist1= otput_final[0]
# idlist2= otput_final[1]
handle1 = Entrez.efetch(db=database, id=idlist1, rettype="gb") 
records1 = SeqIO.read(handle1,"gb")
handle1.close()
s1_=records1.seq
# handle2 = Entrez.efetch(db=database, id=idlist2, rettype="gb") 
# records2 = SeqIO.read(handle2,"gb")
# handle2.close()
# s2_=records2.seq
print(todas_as_seq)
print(s1_)
# print(s2_)

escolha a primeira seq com que quer trabalhar: 3


C:\Users\guilh\anaconda3\lib\site-packages\Bio\Entrez\__init__.py:658: UserWarning: 
Email address is not specified.

To make use of NCBI's E-utilities, NCBI requires you to specify your
email address with each request.  As an example, if your email address
is A.N.Other@example.com, you can specify it as follows:
   from Bio import Entrez
   Entrez.email = 'A.N.Other@example.com'
In case of excessive usage of the E-utilities, NCBI will attempt to contact
a user at the email address provided before blocking access to the
E-utilities.
  warnings.warn(


tipo de molécula com que está a trabalhar: DNA
[Seq('GAGTGGAGATGGCGGAGCTGT'), Seq('CAACACGGGAAACCTCACCC'), Seq('GGGATGGAAGATGGCGGCCTA'), Seq('GGTGGCTAGGGCCGCGCTGGT'), Seq('GTTTGGTGAACTTCTGCCTA')]
GGTGGCTAGGGCCGCGCTGGT


__Goal__ Implementation of sequence manipulation algorithms: transcription, translation, ORFs, extraction of all potential proteins

In [9]:
#function that allows us to associate a codon with an amino acid- Human
def translate_codon(cod):
    #Translates a codon into an aminoacid using an  dictionary with the standard genetic code for Homo Sapiens.
    tc = {"GCT":"A", "GCC":"A", "GCA":"A", "GCG":"A", 
      "TGT":"C", "TGC":"C",
      "GAT":"D", "GAC":"D",
      "GAA":"E", "GAG":"E",
      "TTT":"F", "TTC":"F",
      "GGT":"G", "GGC":"G", "GGA":"G", "GGG":"G",
      "CAT":"H", "CAC":"H",
      "ATA":"I", "ATT":"I", "ATC":"I",
      "AAA":"K", "AAG":"K",
      "TTA":"L", "TTG":"L", "CTT":"L", "CTC":"L", "CTA":"L", "CTG":"L",
      "ATG":"M", "AAT":"N", "AAC":"N",
      "CCT":"P", "CCC":"P", "CCA":"P", "CCG":"P",
      "CAA":"Q", "CAG":"Q",
      "CGT":"R", "CGC":"R", "CGA":"R", "CGG":"R", "AGA":"R", "AGG":"R",
      "TCT":"S", "TCC":"S", "TCA":"S", "TCG":"S", "AGT":"S", "AGC":"S",
      "ACT":"T", "ACC":"T", "ACA":"T", "ACG":"T",
      "GTT":"V", "GTC":"V", "GTA":"V", "GTG":"V",
      "TGG":"W",
      "TAT":"Y", "TAC":"Y",
      "TAA":"_", "TAG":"_", "TGA":"_"}
    if cod in tc: return tc[cod] 
    else: return "This codon does not exist"
#translate_codon("xbt")

In [181]:
while True:
    escolha_de_input= input("You prefer use a personal query or a sequence from Data Base (1-QUERY, 2-SEQUENCE): ")
    if not escolha_de_input.isdigit():
        print("Error: Choose a valid identificator")
        continue
    if escolha_de_input != '1' and escolha_de_input != '2':
        print("Error: Choose an identificator present in the list")

        continue
    else:
        choose_seq_info = escolha_de_input
        break
if choose_seq_info == "1":
    Seq_G= input('Sequence: ')
    Sequence_type= input("Type of molecule (DNA,RNA,PROTEIN): ")
    M=Sequence_type.upper()
    RE=sequence_info(Seq_G,M)
    dic={"1": RE.__len__(), "2":RE.letters(), "3":RE.validate(), "4":RE.contagem(), "5":RE.reverse_complement(),"6":RE.transcription(), "7":RE.translate(),"8":RE.get_orfs_DNA(),"9":RE.get_orfs_Proteins,"10":RE.protsInAA}
    while True:
        f=input("Choose a function between 1-10: ")  
        if not a.isdigit():
            print("Error: Choose a valid identificator")
            continue
        if f != '1' and f != '2' and f != '3' and f != '4' and f != '5' and f != '6' and f != '7' and f != '8' and f != '9' and f != '10':
            print("Error: Choose an identificator present in the list")
            continue
        else:
            result = f
            break
    b=dic[result]
    print(b)

elif choose_seq_info =="2":
    print (Seq_and_length)
    seq3= int(input('choose the numer that correspond to the DB sequence you want to add: '))
    s3=todas_as_seq[seq3]
    print(s3)
    Sequence_type= input("Type of molecule (DNA,RNA,PROTEIN): ")
    M=Sequence_type.upper()
    RE=sequence_info(str(s3),M)
    dic={"1": RE.__len__(), "2":RE.letters(), "3":RE.validate(), "4":RE.contagem(), "5":RE.reverse_complement(),"6":RE.transcription(), "7":RE.translate(),"8":RE.get_orfs_DNA(),"9":RE.get_orfs_Proteins,"10":RE.protsInAA}
    while True:
        f=input("Choose a function between 1-10: ")  
        if not a.isdigit():
            print("Error: Choose a valid identificator")
            continue
        if f != '1' and f != '2' and f != '3' and f != '4' and f != '5' and f != '6' and f != '7' and f != '8' and f != '9' and f != '10':
            print("Error: Choose an identificator present in the list")
            continue
        else:
            result = f
            break
    b=dic[result]
    print(b)
else: print("You must choose 1 or 2")
    


You prefer use a personal query or a sequence from Data Base (1-QUERY, 2-SEQUENCE): 3
Error: Choose an identificator present in the list
You prefer use a personal query or a sequence from Data Base (1-QUERY, 2-SEQUENCE): 2
["('AAADK0043071.1', 21)", "('AAADS0003348.1', 20)", "('AAADW0009410.1', 21)", "('AAADX0043451.1', 21)", "('ABAAM0346030.1', 20)"]
choose the numer that correspond to the DB sequence you want to add: 3
GGTGGCTAGGGCCGCGCTGGT
Type of molecule (DNA,RNA,PROTEIN): dna
Choose a function between 1-10: 4
[('G', 11), ('C', 5), ('T', 4), ('A', 1)]


In [53]:
#Karyna version
while True:
    escolha_de_input= input("You prefer use a personal 1-Query or a 2-Sequence from Data Base (Press 1 or 2): ")
    if not escolha_de_input.isdigit():
        print("Error: Write just the number")
        continue
    if escolha_de_input != '1' and escolha_de_input != '2':
        print("Error: Choose an identificator present in the list (1 or 2)")
        continue
    else:
        choose_seq_info = escolha_de_input
        break
if choose_seq_info == "1":
    Seq_G= input('Sequence: ')
    Sequence_type= input("Type of molecule (DNA, RNA, PROTEIN): ")
    M=Sequence_type.upper()
    RE=sequence_info(Seq_G,M)
    dic={"1": RE.__len__(), "2":RE.letters(), "3":RE.validate(), "4":RE.contagem(), "5":RE.reverse_complement(),"6":RE.transcription(), "7":RE.translate(),"8":RE.get_orfs_DNA(),"9":RE.get_orfs_Proteins,"10":RE.protsInAA}
    while True:
        print("1-length | 2-letter...")
        f=input("Choose a function between 1-10: ")  
        if not f.isdigit():
            print("Error: Choose a valid function number")
            continue
        if int(f) not in range(1,11):
#         if f != '1' and f != '2' and f != '3' and f != '4' and f != '5' and f != '6' and f != '7' and f != '8' and f != '9' and f != '10':
            print("Error: Choose an identificator present in the list")
            continue
        else:
            result = f
            break
    b=dic[result]
    print(b)

elif choose_seq_info =="2":
#     print (Seq_and_length)
    password = input("Enter password: ")
    db=DB(password)
    count = 0
    all_info=[]
    for index, value in enumerate(db.extract_sequences()):
            selected_seq = []
            selected_seq.append(count)
            selected_seq.append(''.join(db.extract_ids()[index]))
            selected_seq.append(len(''.join(value)))
            all_info.append(selected_seq)
            count += 1
    print("\nLabels for the output [1st index, 2nd index, 3rd index]: \n[Identificator, Id of the Gene, Length sequence]\n\n")
    print(all_info)
    identificator = [str(i[0]) for i in all_info]
    while True:
        seq3= input('Choose the identificator (1st index of each previous list) that corresponds to the DB sequence: ')
        if seq3 in identificator:
#             seq3_int=int(seq3)
            s3=''.join(db.extract_sequences()[int(seq3)])
            print("\nHere is the sequence from DataBase: ", s3)
#             Sequence_type= input("Type of molecule (DNA,RNA,PROTEIN): ")
            Sequence_type= "DNA"
#             M=Sequence_type.upper()
            RE=sequence_info(str(s3),Sequence_type)
            dic={"1": RE.__len__(), "2":RE.letters(), "3":RE.validate(), "4":RE.contagem(), "5":RE.reverse_complement(),"6":RE.transcription(), "7":RE.translate(),"8":RE.get_orfs_DNA(),"9":RE.get_orfs_Proteins,"10":RE.protsInAA}
            break
        else:        
            print("Error: Choose a valid identificator")
            continue
    while True:
        f=input("\nChoose a function between 1-10: ")  
        if not f.isdigit():
            print("Error: Choose a valid function number")
            continue
        if int(f) not in range(1,11):
            print("Error: Choose an identificator present in the list")
            continue
        else:
            result = f
            break
    b=dic[result]
    print(b)
else: print("You must choose 1 or 2")
   

You prefer use a personal 1-Query or a 2-Sequence from Data Base (Press 1 or 2): 2
Enter password: 20221207

Labels for the output [1st index, 2nd index, 3rd index]: 
[Identificator, Id of the Gene, Length sequence]


[[0, 'A00829.1', 50], [1, 'AAADK0043071.1', 21], [2, 'AAADS0003348.1', 20], [3, 'AAADW0009410.1', 21], [4, 'AAADX0043451.1', 21], [5, 'AAVX01203288.1', 50], [6, 'ABAAM0346030.1', 20], [7, 'BAAF03073813.1', 50], [8, 'DH405380.1', 50], [9, 'DS196524.1', 50], [10, 'EC923692.1', 50], [11, 'EC966285.1', 50], [12, 'M97702.1', 50], [13, 'X65482.1', 50], [14, 'X68316.1', 50]]
Choose the identificator (1st index of each previous list) that corresponds to the DB sequence: 0

Here is the sequence from DataBase:  TACCTTGTCCGAAAAGGAAAGACAAATCAAGAAGCAAACTGCTTTGGTTG

Choose a function between 1-10: 0
Error: Choose an identificator present in the list

Choose a function between 1-10: a
Error: Choose a valid function number

Choose a function between 1-10: 1
50


In [7]:
#a class that allows to accomplish the objective
class sequence_info:
    #"building block", where you always have to have the sequence you are analyzing and the type of molecule: DNA, RNA, Protein
    def __init__(self,sequence, sequence_type):
        if sequence_type not in ["DNA","RNA","PROTEIN"]:
            raise ValueError("Invalid sequence type")
        self.sequence=sequence.upper() 
        self.sequence_type=sequence_type
    def __len__(self):
        return len(self.sequence)
    def __getitem__(self,n):
        return self.sequence[n]
    def __getslice__(self,i,j):
        return self.sequence[i:j]
    def __str__(self):
        return self.sequence
    def sequence_type(self):
        return self.sequence_type
    def letters(self):
        #each letter possible to have for DNA, RNA , PROTEIN for later to be able to validate the information about the type of molecule corresponding to the sequence
        if (self.sequence_type=="DNA"): return "ACGT"
        elif(self.sequence_type=="RNA"):return "ACGU"
        elif(self.sequence_type=="PROTEIN"): return "ACDEFGHIKLMNPQRSTVWY"
        else: raise ValueError("ERROR")
    def validate(self): #validates information about the type of molecule associated with the sequence
        letras=self.letters()
        res=True
        i=0
        while i<len(self.sequence) and res:
            if self.sequence[i] not in letras: res = False
            else: i +=1
        if res==False:
            raise ValueError("Put the right sequence_type")
        else: return res 
    def contagem(self):
        contagem={}
        for x in self.sequence:
            contagem[x]=contagem.get(x,0)+1
        return sorted(contagem.items(), key=lambda x: x[1], reverse=True)      
    def reverse_complement(self):#falta comprovar isto
        if (self.validate() == False) or (self.sequence_type in ["RNA","PROTEIN"]) :return "only want to do reverse complement with DNA"
        t=self.sequence.replace("T","a")
        g=t.replace("G","c")
        c=g.replace("C","g")
        a=c.replace("A","t")
        reverse_comp=a[::-1]
        return reverse_comp.upper()    
    def transcription(self):
        if (self.validate() == False) or (self.sequence_type != "DNA"):return "only want to do transcription with DNA"
        else: return  sequence_info(self.sequence.replace("T","U"),"RNA")
    def translate(self):
        if (self.validate() == False) or (self.sequence_type != "DNA"): return "only want to do translation with DNA"
        sequence_aa=""
        for x in range(0, len(self.sequence)-2,3):#3 by 3
            codon=self.sequence[x:x+3]
            sequence_aa += translate_codon(codon)
        return sequence_aa
    def get_orfs_DNA(self): #6 possible orfs
        if (self.validate() == False) or (self.sequence_type != "DNA"): return "only want to do DNA orf"
        inverse_sequence=self.sequence[::-1]
        return ",".join([self.sequence[p:]for p in range(3)] + [inverse_sequence[p:] for p in range(3)]) 
    def get_orfs_Proteins(self): # translation of the 6 possible dna orfs
        if (self.validate() == False) or (self.sequence_type != "DNA"): return "just work from DNA orf"
        inverse_sequence=self.sequence[::-1]
        orfs_protein=[]
        for p in range(3):
            orfs_protein.append(sequence_info(self.sequence[p:],self.sequence_type).translate())
            orfs_protein.append(sequence_info(inverse_sequence[p:],self.sequence_type).translate())
        return ",".join(orfs_protein)
    def protsInAA(self):#gives all possible proteins of an amino acid sequence( from M to _)
        seq_aa = self.translate()
        prots_act = []
        prots = []
        for aa in seq_aa:
            if aa == '_':
                if prots_act:
                    for p in prots_act:
                        prots.append(p)
                    prots_act = []
            else:
                if aa == 'M':
                    prots_act.append('')
                for i in range(len(prots_act)):
                    prots_act[i] += aa
        return ",".join(prots) 
 #if name=main é para conseguir importar direitinho os nodulos ou seja quanto estou neste ficheiro o name =main e ele faz a classe se for para outro ficheiro e importar a classe ele faz na mesma a classe sem repetir tufo o que aqui tenho
# https://www.youtube.com/watch?v=3dHyS1W4TIE
#if __name__ == "__main__":
    #s1_="AA??"
    s1 = sequence_info("ACTGCTG","PROTEIN")
    
    #s2 = sequence_info("MKVVLSVQERSVVSLL_", "PROTEIN")
    #s3 = sequence_info("ATTTTBTT","PROTEIN")
    #print("sequence 1 is {}? {}".format(s1.sequence_type,s1.validate()))
    s3 = s1.validate()
    print(s3)
    #s4 = s1.reverse_complement()
    #print(s4)
#     s5= s1.translate()
#     print(s5)
    #s6=s1.get_orfs_DNA()
    #print(s6)
    #s8=s1.get_orfs_Proteins()
    #print(s8)
#     s9=s1.protsInAA()
#     print(s9)
#     s10=s1.contagem()
#     print(s10)
    #s11=s1.__len__()
    #print(s11)
    #s12=s1.letters()
    #print(s12)
    

    
    

In [174]:
import unittest
class TestSequenceInfo(unittest.TestCase):
    def test_init(self):
        seq = sequence_info("AGCT", "DNA")
        self.assertEqual(seq.sequence, "AGCT")
        self.assertEqual(seq.sequence_type, "DNA")
        
        seq=sequence_info("AGCU","RNA")
        self.assertEqual(seq.sequence, "AGCU")
        self.assertEqual(seq.sequence_type, "RNA")
        
        with self.assertRaises(ValueError) as error:
            seq = sequence_info("AA??", "DNS")
        self.assertEqual(error.exception.args[0],"Invalid sequence type")
        
        seq = sequence_info("AA??", "DNA")
        self.assertEqual(seq.sequence, "AA??")
        self.assertEqual(seq.sequence_type, "DNA")

    def test_len(self):
        seq = sequence_info("AGCT", "DNA")
        self.assertEqual(len(seq), 4)
        
        seq = sequence_info("AGCU","RNA")
        self.assertEqual(len(seq), 4)
        
        seq=sequence_info("AA??","DNA")
        self.assertEqual(len(seq),4)

    def test_getitem(self):
        seq = sequence_info("AGCT", "DNA")
        self.assertEqual(seq[0], "A")
        
        seq = sequence_info("AGCU","RNA")
        self.assertEqual(seq[0], "A")
        
        seq=sequence_info("AA??","DNA")
        self.assertEqual(seq[0],"A")
        
    def test_getslice(self):
        seq = sequence_info("AGCT", "DNA")
        self.assertEqual(seq[0:2], "AG")
        
        seq = sequence_info("AGCU","RNA")
        self.assertEqual(seq[0:2], "AG")
        
        seq=sequence_info("AA??","DNA")
        self.assertEqual(seq[1:3],"A?")

    def test_str(self):
        seq = sequence_info("AGCT", "DNA")
        self.assertEqual(str(seq), "AGCT")
        
        seq = sequence_info("AGCU","RNA")
        self.assertEqual(str(seq), "AGCU")
        
        seq=sequence_info("AA??","DNA")
        self.assertEqual(str(seq), "AA??")

    def test_sequence_type(self):
        seq = sequence_info("AGCT", "DNA")
        self.assertEqual(seq.sequence_type, "DNA")
        
        seq=sequence_info("AGCU","RNA")
        self.assertEqual(seq.sequence_type, "RNA")
        
        seq=sequence_info("AA??","DNA")
        self.assertEqual(seq.sequence_type, "DNA")
        
    def test_letters(self):
        seq = sequence_info("AGCT", "DNA")
        self.assertEqual(seq.letters(), "ACGT")
        
        seq=sequence_info("AGCU","RNA")
        self.assertEqual(seq.letters(), "ACGU")
        
        seq = sequence_info("AA??", "DNA")
        self.assertEqual(seq.letters(), "ACGT")

    def test_validate(self):
        seq = sequence_info("AGCT", "DNA")
        self.assertTrue(seq.validate())
        
        seq = sequence_info("AGCU","RNA")
        self.assertTrue(seq.validate())
        
        seq = sequence_info("AA??", "DNA")
        try:
            seq.validate()
        except ValueError as ve:
            assert str(ve) == "Put the right sequence_type", f"Unexpected error message: {str(ve)}"
        else:
            assert False, "ValueError not raised"

    def test_contagem(self):
        seq = sequence_info("AGCT", "DNA")
        self.assertEqual(seq.contagem(), [("A", 1), ("G", 1), ("C", 1), ("T", 1)])
        
        seq = sequence_info("AGCU", "RNA")
        self.assertEqual(seq.contagem(), [("A", 1), ("G", 1), ("C", 1), ("U", 1)])
        
        seq= sequence_info("AA??","DNA")
        self.assertEqual(seq.contagem(),[("A",2),("?",2)])

    def test_reverse_complement(self):
        seq = sequence_info("AGCT", "DNA")
        self.assertEqual(seq.reverse_complement(), "AGCT")
        
        seq = sequence_info("AGCU", "RNA")
        self.assertEqual(seq.reverse_complement(), "only want to do reverse complement with DNA")
        
        seq = sequence_info("AA??", "DNA")
        try:
            seq.reverse_complement()
        except ValueError as ve:
            assert str(ve) == "Put the right sequence_type", f"Unexpected error message: {str(ve)}"
        else:
            assert False, "ValueError not raised"

    def test_transcription(self):
        seq = sequence_info("AGCT", "DNA")
        self.assertEqual(seq.transcription().sequence, "AGCU")
        seq = sequence_info("AGCU", "RNA")
        self.assertEqual(seq.transcription(), "only want to do transcription with DNA")
        
        seq = sequence_info("AA??", "DNA")
        try:
            seq.transcription()
        except ValueError as ve:
            assert str(ve) == "Put the right sequence_type", f"Unexpected error message: {str(ve)}"
        else:
            assert False, "ValueError not raised"

    def test_translate(self):
        seq = sequence_info("AGCT", "DNA")
        self.assertEqual(seq.translate(), "S")
        
        seq = sequence_info("AGCU", "RNA")
        self.assertEqual(seq.translate(), "only want to do translation with DNA")
        
        seq = sequence_info("AA??", "DNA")
        try:
            seq.translate()
        except ValueError as ve:
            assert str(ve) == "Put the right sequence_type", f"Unexpected error message: {str(ve)}"
        else:
            assert False, "ValueError not raised"

    def test_get_orfs_DNA(self):
        seq = sequence_info("AGCT", "DNA")
        self.assertEqual(seq.get_orfs_DNA(), "AGCT,GCT,CT,TCGA,CGA,GA")
        
        seq = sequence_info("AGCU", "RNA")
        self.assertEqual(seq.get_orfs_DNA(), "only want to do DNA orf")
        
        seq = sequence_info("AA??", "DNA")
        try:
            seq.get_orfs_DNA()
        except ValueError as ve:
            assert str(ve) == "Put the right sequence_type", f"Unexpected error message: {str(ve)}"
        else:
            assert False, "ValueError not raised"

    def test_get_orfs_Proteins(self):
        seq = sequence_info("AGCT", "DNA")
        self.assertEqual(seq.get_orfs_Proteins(), "S,S,A,R,,")
        
        seq = sequence_info("AGCU", "RNA")
        self.assertEqual(seq.get_orfs_Proteins(), "just work from DNA orf")
        
        seq = sequence_info("AA??", "DNA")
        try:
            seq.get_orfs_Proteins()
        except ValueError as ve:
            assert str(ve) == "Put the right sequence_type", f"Unexpected error message: {str(ve)}"
        else:
            assert False, "ValueError not raised"

    def test_protsInAA(self):
        seq = sequence_info("AGCT", "DNA")
        self.assertEqual(seq.protsInAA(), "")
        
        seq = sequence_info("AGCU", "RNA")
        self.assertEqual(seq.protsInAA(), "")
        
        seq = sequence_info("AA??", "DNA")
        try:
            seq.protsInAA()
        except ValueError as ve:
            assert str(ve) == "Put the right sequence_type", f"Unexpected error message: {str(ve)}"
        else:
            assert False, "ValueError not raised"

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

...............
----------------------------------------------------------------------
Ran 15 tests in 0.012s

OK
